In [1]:
import requests, json
import pandas as pd
from urllib.parse import quote

In [15]:
pop = pd.read_csv("data/test2.csv", index_col="Unnamed: 0")
pop

,지역명,측정소명,주소,운영기관,설치년도,위도,경도,구,중복여부
0,서울,강남구,서울 강남구 학동로 426 강남구청 별관 1동,서울특별시보건환경연구원,1978.0,37.517968,127.047060,강남구,False
1,서울,강동구,서울 강동구 구천면로 42길 59 천호1동 주민센터,서울특별시보건환경연구원,1997.0,37.545021,127.136799,강동구,False
2,서울,강북구,서울 강북구 삼양로 139길 49 우이동 주민센터,서울특별시보건환경연구원,1997.0,37.647934,127.011870,강북구,False
3,서울,강서구,서울 강서구 강서로 45 다길 71 화곡3동 푸른들청소년도서관,서울특별시보건환경연구원,1992.0,37.544670,126.835179,강서구,False
4,서울,관악구,서울 관악구 신림동길 14 신림동 주민센터,서울특별시보건환경연구원,1979.0,37.487396,126.927114,관악구,False
...,...,...,...,...,...,...,...,...,...
520,제주,연동,제주 제주시 연동 322-1 번지 제주특별자치도청 제2청사 제주특별자치도청 제2청사 옥상,제주특별자치도보건환경연구원,2006.0,33.489045,126.500458,제주시,True
521,제주,이도동,제주 제주시 이도2동 1176-1(제주시청) 제주시청,제주특별자치도보건환경연구원,1994.0,33.481576,126.520156,제주시,True
522,제주,조천읍,제주특별자치도 제주시 조천읍 조천18길 11-1 조천읍 체육관,제주특별자치도보건환경연구원,2020.0,33.539536,126.643719,제주시,True
523,제주,한림읍,제주특별자치도 제주시 한림읍 한림중앙로 71-9 한림읍 체육관,제주특별자치도보건환경연구원,2020.0,33.409692,126.268544,제주시,True


In [27]:
# 시군구 값을 기본값으로 가져감
# 고성군 - 고성(강원), 고성(경남)
# 광역시 - 대전 서구, 서울 중구, 서울 용산, 부산 해운대, 세종
# 행정구 - 포항 북구, 수원 장안, 용인 수지, 고양 일산동, 창원 합포, 창원 회원
tmp_gu_dict = {
    '수원': ['장안구', '권선구', '팔달구', '영통구'], 
    '성남': ['수정구', '중원구', '분당구'], 
    '안양': ['만안구', '동안구'], 
    '안산': ['상록구', '단원구'], 
    '고양': ['덕양구', '일산동구', '일산서구'], 
    '용인': ['처인구', '기흥구', '수지구'], 
    '청주': ['상당구', '서원구', '흥덕구', '청원구'], 
    '천안': ['동남구', '서북구'], 
    '전주': ['완산구', '덕진구'], 
    '포항': ['남구', '북구'], 
    '창원': ['의창구', '성산구', '진해구', '마산합포구', '마산회원구']
}

In [28]:
metro_list = ['서울','부산','대구','인천','대전','광주','울산']

In [29]:
si_name = [None] * len(pop)
for i in pop.index:
    if pop.지역명[i] in metro_list:
        if len(pop.구[i]) == 2:
            si_name[i] = pop.지역명[i]+ ' ' + pop.구[i]          # 서울 중구, 대전 서구
        else:       # 3~ 4글자
            si_name[i] = pop.지역명[i] + ' ' + pop.구[i][:-1]     # 서울 용산, 서울 영등포
    else:
        if pop.구[i][:-1] == '고성':
            if pop.지역명[i] == '강원':
                si_name[i] = '고성(강원)'
            else:
                si_name[i] = '고성(경남)'
        else:
            si_name[i] = pop.구[i][:-1]     # 세종, 광명, 의정부
        for key, value in tmp_gu_dict.items():
            if pop.구[i] in value:              # 구의 이름이 행정구이면
                if len(pop.구[i]) == 2:
                    si_name[i] = key + ' ' + pop.구[i]      # 포항 북구, 포항 남구
                elif len(pop.구[i]) == 5:
                    si_name[i] = key + ' ' + pop.구[i][2:-1]    #창원 합포, 창원 회원
                else: # 3~4 글자
                    si_name[i] = key + ' ' + pop.구[i][:-1]     # 수원 장안, 고양 일산동

In [30]:
si_name

['서울 강남',
 '서울 강동',
 '서울 강북',
 '서울 강서',
 '서울 관악',
 '서울 광진',
 '서울 구로',
 '서울 금천',
 '서울 노원',
 '서울 도봉',
 '서울 동대문',
 '서울 동작',
 '서울 마포',
 '서울 서대문',
 '서울 서초',
 '서울 성동',
 '서울 성북',
 '서울 송파',
 '서울 양천',
 '서울 영등포',
 '서울 용산',
 '서울 은평',
 '서울 성동',
 '서울 중구',
 '서울 중랑',
 '여주',
 '가평',
 '광주',
 '평택',
 '수원시권선',
 '남양주',
 '안산시단원',
 '의왕',
 '김포',
 '광주',
 '안성',
 '과천',
 '수원시영통',
 '구리',
 '남양주',
 '파주',
 '용인시기흥',
 '용인시처인',
 '화성',
 '부천',
 '성남시수정',
 '군포',
 '안산시단원',
 '여주',
 '시흥',
 '구리',
 '화성',
 '용인시처인',
 '시흥',
 '하남',
 '시흥',
 '남양주',
 '용인시처인',
 '남양주',
 '과천',
 '동두천',
 '성남시수정',
 '안산시상록',
 '화성',
 '안성',
 '의왕',
 '안산시상록',
 '안양시동안',
 '이천',
 '평택',
 '김포',
 '군포',
 '성남시중원',
 '화성',
 '화성',
 '포천',
 '가평',
 '부천',
 '광명',
 '평택',
 '의정부',
 '성남시분당',
 '용인시수지',
 '시흥',
 '고양시일산동',
 '고양시덕양',
 '하남',
 '수원시팔달',
 '안양시만안',
 '안양시만안',
 '평택',
 '양평',
 '연천',
 '수원시영통',
 '남양주',
 '오산',
 '부천',
 '광주',
 '남양주',
 '양평',
 '화성',
 '파주',
 '성남시분당',
 '안산시단원',
 '안산시단원',
 '김포',
 '의정부',
 '의정부',
 '용인시처인',
 '수원시팔달',
 '포천',
 '시흥',
 '이천',
 '연천',
 '시흥',
 '성남시분당',
 '고양시일산서',
 '안성

In [31]:
len(si_name)

525

In [32]:
pop['시군구'] = si_name
pop

,지역명,측정소명,주소,운영기관,설치년도,위도,경도,구,중복여부,시군구
0,서울,강남구,서울 강남구 학동로 426 강남구청 별관 1동,서울특별시보건환경연구원,1978.0,37.517968,127.047060,강남구,False,서울 강남
1,서울,강동구,서울 강동구 구천면로 42길 59 천호1동 주민센터,서울특별시보건환경연구원,1997.0,37.545021,127.136799,강동구,False,서울 강동
2,서울,강북구,서울 강북구 삼양로 139길 49 우이동 주민센터,서울특별시보건환경연구원,1997.0,37.647934,127.011870,강북구,False,서울 강북
3,서울,강서구,서울 강서구 강서로 45 다길 71 화곡3동 푸른들청소년도서관,서울특별시보건환경연구원,1992.0,37.544670,126.835179,강서구,False,서울 강서
4,서울,관악구,서울 관악구 신림동길 14 신림동 주민센터,서울특별시보건환경연구원,1979.0,37.487396,126.927114,관악구,False,서울 관악
...,...,...,...,...,...,...,...,...,...,...
520,제주,연동,제주 제주시 연동 322-1 번지 제주특별자치도청 제2청사 제주특별자치도청 제2청사 옥상,제주특별자치도보건환경연구원,2006.0,33.489045,126.500458,제주시,True,제주
521,제주,이도동,제주 제주시 이도2동 1176-1(제주시청) 제주시청,제주특별자치도보건환경연구원,1994.0,33.481576,126.520156,제주시,True,제주
522,제주,조천읍,제주특별자치도 제주시 조천읍 조천18길 11-1 조천읍 체육관,제주특별자치도보건환경연구원,2020.0,33.539536,126.643719,제주시,True,제주
523,제주,한림읍,제주특별자치도 제주시 한림읍 한림중앙로 71-9 한림읍 체육관,제주특별자치도보건환경연구원,2020.0,33.409692,126.268544,제주시,True,제주


In [34]:
pop.to_csv('data/test3.csv')
pop

,지역명,측정소명,주소,운영기관,설치년도,위도,경도,구,중복여부,시군구
0,서울,강남구,서울 강남구 학동로 426 강남구청 별관 1동,서울특별시보건환경연구원,1978.0,37.517968,127.047060,강남구,False,서울 강남
1,서울,강동구,서울 강동구 구천면로 42길 59 천호1동 주민센터,서울특별시보건환경연구원,1997.0,37.545021,127.136799,강동구,False,서울 강동
2,서울,강북구,서울 강북구 삼양로 139길 49 우이동 주민센터,서울특별시보건환경연구원,1997.0,37.647934,127.011870,강북구,False,서울 강북
3,서울,강서구,서울 강서구 강서로 45 다길 71 화곡3동 푸른들청소년도서관,서울특별시보건환경연구원,1992.0,37.544670,126.835179,강서구,False,서울 강서
4,서울,관악구,서울 관악구 신림동길 14 신림동 주민센터,서울특별시보건환경연구원,1979.0,37.487396,126.927114,관악구,False,서울 관악
...,...,...,...,...,...,...,...,...,...,...
520,제주,연동,제주 제주시 연동 322-1 번지 제주특별자치도청 제2청사 제주특별자치도청 제2청사 옥상,제주특별자치도보건환경연구원,2006.0,33.489045,126.500458,제주시,True,제주
521,제주,이도동,제주 제주시 이도2동 1176-1(제주시청) 제주시청,제주특별자치도보건환경연구원,1994.0,33.481576,126.520156,제주시,True,제주
522,제주,조천읍,제주특별자치도 제주시 조천읍 조천18길 11-1 조천읍 체육관,제주특별자치도보건환경연구원,2020.0,33.539536,126.643719,제주시,True,제주
523,제주,한림읍,제주특별자치도 제주시 한림읍 한림중앙로 71-9 한림읍 체육관,제주특별자치도보건환경연구원,2020.0,33.409692,126.268544,제주시,True,제주


In [ ]:
pop.drop_duplicates(subset='시군구')

In [ ]:
import folium

map = folium.Map(location=[37.56682, 126.97865], zoom_start=13)  # 중심 위치

for i in df3.index:
    if i in df3.index and pd.notnull(df3['위도'][i]) and pd.notnull(df3['경도'][i]):  # 해당 인덱스가 있는지 체크 후 None 값인 경우 건너뜀
        folium.Marker(
            location=[df3['위도'][i], df3['경도'][i]],  # WGS84 좌표계: 위도(latitude), 경도(longitude)
            tooltip=df3['측정소명'][i],
            popup=folium.Popup(df3['주소'][i], max_width=200)
        ).add_to(map)

title_html = '<h3 align="center" style="font-size:20px">측정소 위치</h3>'
map.get_root().html.add_child(folium.Element(title_html))

map


In [ ]:
df3.to_csv("data/측정소좌표(시군구구분_중복제거).csv")
df3

,지역명,측정소명,주소,운영기관,설치년도,위도,경도,구
0,서울,강남구,서울 강남구 학동로 426 강남구청 별관 1동,서울특별시보건환경연구원,1978.0,37.517968,127.04706,None
